In [ ]:
'''
Setup
1. Import Library
2. Import File
3. Cleaning data
4. Feature Engineering
5. Buat Model
6. Tes Model
'''

In [1]:
#Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print("Setup done")

Setup done


In [15]:
#Import File
data= pd.read_csv("C:/Users/visco/Downloads/Housing Prize/train.csv")
print(data.shape)
data.iloc[:,20:30]

(1460, 81)


,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation
0,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc
1,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock
2,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc
3,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil
4,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc
...,...,...,...,...,...,...,...,...,...,...
1455,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc
1456,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock
1457,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone
1458,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock


In [21]:
#Liat data hilang
data_hilang = data.isnull().sum()
data_hilang[70:81]

ScreenPorch         0
PoolArea            0
PoolQC           1453
Fence            1179
MiscFeature      1406
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
dtype: int64

In [26]:
#Hapus data Hilang
data_bersih = data.dropna(axis= 1) #Hapus kolom yang punya missing values
print(f'Columns Before Data Cleaning: {data.shape[1]}')
print(f'Columns After Data Cleaning: {data_bersih.shape[1]}')


Columns Before Data Cleaning: 81
Columns After Data Cleaning: 62
